In [24]:
from pyraf import iraf
from iraf import obsutil
import subprocess
from astropy.table import Table
import os
import csv
#in the furture make packages and import common functions
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt


In [34]:
imagename = '/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits[2]'

In [35]:
def makeConfig(valList,defaultDir='/home/mj1e16/sextractor/sextractor-master/config/',attributeList=['DETECT_THRESH','DETECT_MINAREA','BACK_FILTERSIZE','BACK_SIZE']):
    
    with open(defaultDir+'/default.sex','r') as f:
        data  = f.read()
    
    catLocFinder = 'CATALOG_NAME'
    catname = 'test_PSF.cat'
    nameLoc = data.find(catLocFinder) + len(catLocFinder)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' '+catname+' ' + data[endLoc:]
    data = newData 
        
    for x in range(len(valList)):
        nameLoc = data.find(attributeList[x]) + len(attributeList[x])
        endLoc = data[nameLoc:].find('#') + nameLoc
        newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
        data = newData    
    
    
    confName = 'default_PSF.sex'
    with open(defaultDir+confName,'w') as f:
        f.write(data)
    return(confName,catname)

In [36]:
def findObjects(imagename,confName,catName,defaultDir='/home/mj1e16/sextractor/sextractor-master/config/'):
    
    os.chdir(defaultDir)
    subprocess.call(['sex',imagename,'-c',confName])
    assoc = Table.read(catName,format='ascii.sextractor')
    
    return assoc

In [57]:
def findPSF(outTable,imageName):
    df = outTable.to_pandas()
    dflen = len(df)
    top10 = int(dflen/10)
    newdf = df.nlargest(top10,'FLUX_ISO')
    df2 = newdf[50:top10]
    
    os.chdir('/home/mj1e16/iraf/')
    length = len(df2)
    nsegs = 25
    segment = length/nsegs
    PSF = []
    for segs in range(3): #nsegs):
        low = segs*segment
        high = low+segment
        regiondf = df2.loc[(df['Y_IMAGE'] >= low) & (df['Y_IMAGE'] <= high)]
        xcoords = regiondf['X_IMAGE'].tolist()
        ycoords = regiondf['Y_IMAGE'].tolist()
        with open('starlist','w') as f:
            writer = csv.writer(f,delimiter=' ')
            writer.writerows(zip(xcoords,ycoords))
        psfmeasure(imageName,coords='markall',logfile='logfile'.format('imagename',segs),imagecur='starlist') 
        with open('logfile','r') as f:
            bigstring = f.read()
        finder = 'Average full width at half maximum (FWHM) of '
        loc1 = bigstring.find(finder) +len(finder)
        loc2 = bigstring[loc1:].find('\n')
        print(loc1,loc2)
        PSF.append(bigstring[loc1:loc1+loc2])
    return PSF

In [58]:
findPSF(outTable,imagename)

** Select stars to measure with 'm' and finish with 'q'.
** Additional options are '?', 'g', and :show.
NOAO/IRAF V2.16 mj1e16@uos-201660 Mon 20:13:48 10-Jun-2019

          Image  Column    Line     Mag    FWHM   Ellip      PA SAT
/data/mj1e16/ke  771.76   20.63    0.00   1.674    0.22      55
                 262.46   23.87    1.12   1.597    0.41      58
                1103.37   27.22    0.12  21.475    0.23     -60

  Average full width at half maximum (FWHM) of 9.5366
(368, 6)
** Select stars to measure with 'm' and finish with 'q'.
** Additional options are '?', 'g', and :show.
NOAO/IRAF V2.16 mj1e16@uos-201660 Mon 20:13:49 10-Jun-2019

          Image  Column    Line     Mag    FWHM   Ellip      PA SAT
/data/mj1e16/ke  795.14   39.94    0.21   2.156    0.29      71
                 954.77   30.30    0.59   1.471    0.23      54
                 636.38   32.52    1.80   1.354    0.35      75
                 749.01   43.55    1.92   1.490    0.28      64
                1092.33 

['3.6589', '3.6589', '3.6589']

In [37]:
names = makeConfig([5,10,3,64])
outTable = findObjects(imagename,names[0],names[1])

In [38]:
df = outTable.to_pandas()
newdf = df.nlargest(1050,'FLUX_ISO')

In [39]:
df2 = newdf[50:1050]

In [45]:
dfbottomtotop = []
os.chdir('/home/mj1e16/iraf/')
length = len(df2)
nsegs = 25
segment = length/nsegs

for segs in range(nsegs):
    low = segs*segment
    high = low+segment
    regiondf = df2.loc[(df['Y_IMAGE'] >= low) & (df['Y_IMAGE'] <= high)]
    xcoords = regiondf['X_IMAGE'].tolist()
    ycoords = regiondf['Y_IMAGE'].tolist()
    with open('starlist','w') as f:
        writer = csv.writer(f,delimiter=' ')
        writer.writerows(zip(xcoords,ycoords))
    psfmeasure('kplr2009115002613_ffi-cal.fits[2]',coords='markall',logfile='logfile_{}_{}'.format('imagename',segs),imagecur='starlist') 


** Select stars to measure with 'm' and finish with 'q'.
** Additional options are '?', 'g', and :show.
NOAO/IRAF V2.16 mj1e16@uos-201660 Mon 20:01:47 10-Jun-2019

          Image  Column    Line     Mag    FWHM   Ellip      PA SAT
kplr20091150026  954.77   30.30    0.59   1.471    0.23      54
                 636.38   32.52    1.80   1.354    0.35      75
                 771.76   20.63    2.15   1.672    0.20      54
                 934.28   34.87    2.44   1.308    0.31      54
                 157.70   28.33    0.00   1.999    0.15      71
                1104.99   36.19    4.83   1.293    0.44       4
                 662.95   33.38    2.71   1.942    0.56     -14
                 367.21   29.42    3.22   1.552    0.41       5
                 262.46   23.87    3.27   1.594    0.41      58
                1001.37   35.09    3.51   1.226    0.35      51
                1031.08   33.89    3.57   1.334    0.30      54
                1116.13   14.11    6.72  22.015    0.94     -52


In [9]:
from iraf import psfmeasure

In [10]:
os.chdir('/home/mj1e16/iraf/')

In [20]:
psfmeasure('kplr2009115002613_ffi-cal.fits[44]',coords='markall',logfile='logfile',imagecur='starlist') #)

** Select stars to measure with 'm' and finish with 'q'.
** Additional options are '?', 'g', and :show.
NOAO/IRAF V2.16 mj1e16@uos-201660 Mon 19:56:52 10-Jun-2019

          Image  Column    Line     Mag    FWHM   Ellip      PA SAT
kplr20091150026  684.11  500.74    3.64   1.407    0.17     -69
                 674.83  732.79    0.00   3.940    0.84     -88
                 153.29  644.73    0.24   3.406    0.70     -89

  Average full width at half maximum (FWHM) of 3.6589
